In [2]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns
# modeling
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
#from xgboost import XGBoost
import warnings

ImportError: cannot import name 'XGBoostRegressor' from 'xgboost' (e:\ML ALL\MLProject\venv\lib\site-packages\xgboost\__init__.py)

## Importing dataset 

In [1]:
df = pd.read_csv('data/data.csv')

NameError: name 'pd' is not defined